In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_ut.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,886823,4440597205,4/27/20,883,PPP,ALLISON BAVER ENTERTAINMENT LLC,6219 S. Saunter Lane,SALT LAKE CITY,UT,84123.0,...,999 Other,5121 Motion Picture and Video Industries,5121.0,UTAH DISTRICT OFFICE,e $5-10 million,250 to 499,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,NaN
1,886824,5333267308,4/30/20,883,PPP,"INTERMOUNTAIN EMPLOYMENT SERVICE, INC.",450 East 1000 North,NORTH SALT LAKE,UT,84054.0,...,999 Other,5613 Employment Services,5613.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,NaN
2,886825,7850817707,5/1/20,883,PPP,CENTRAL UTAH CLINIC PC,1055 North 500 West,Provo,UT,84604.0,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"1055 North 500 West, Provo, UT",40.248562,-111.668402,NaN
3,886826,8803828610,3/25/21,883,PPP,WOLVERINE FUELS HOLDING LLC,9815 S Monroe St Ste 203,Sandy,UT,84070.0,...,2121 Coal Mining,2121 Coal Mining,2121.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,NaN
4,886827,9607187010,4/9/20,883,PPP,"SIZZLING CAESARS, LLC",348 E WINCHESTER ST,SALT LAKE CITY,UT,84107.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9290,896113,9048067208,4/28/20,883,PPP,PRO TOWER MAINTENANCE LLC,676 LAVA POINTE DR #50,SAINT GEORGE,UT,84770.0,...,999 Other,2383 Building Finishing Contractors,2383.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"676 LAVA POINTE DR #50, SAINT GEORGE, UT",37.076327,-113.599085,NaN
9291,896114,9342157005,4/9/20,883,PPP,"EDWARDS AUTOMOTIVE, INC",61 E GORDON AVE,SALT LAKE CITY,UT,84107.0,...,999 Other,8111 Automotive Repair and Maintenance,8111.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"61 E GORDON AVE, SALT LAKE CITY, UT",40.683543,-111.889893,NaN
9292,896115,9527478402,2/17/21,883,PPS,RUBYSNAP LLC,756 S 300 W,Salt Lake City,UT,84101.0,...,999 Other,3118 Bakeries and Tortilla Manufacturing,3118.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"756 S 300 W, Salt Lake City, UT",40.752633,-111.900108,NaN
9293,896116,9736917000,4/9/20,883,PPP,"CODY EKKER CONSTRUCTION, INC.",412 E. 3200 N.,LEHI,UT,84043.0,...,999 Other,2362 Nonresidential Building Construction,2362.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"412 E. 3200 N., LEHI, UT",40.427645,-111.842536,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,4440597205,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047
1,5333267308,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968
2,7850817707,"1055 North 500 West, Provo, UT",40.248562,-111.668402
3,8803828610,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991
4,9607187010,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/UT_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [27]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [28]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

<h4>FIPS Script

In [7]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [8]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
executable_path = {'executable_path': ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', **executable_path, options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('49')] #UT
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/9295 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [9]:
geo_id_list =geo_id 

Length Check

In [10]:
print(len(geo_id_list))

9295


Matching FIPS to DataFrame

In [11]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:25062]

In [12]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,4440597205,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,[490351140002011]
1,5333267308,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,[490111270024066]
2,7850817707,"1055 North 500 West, Provo, UT",40.248562,-111.668402,[490490020013000]
3,8803828610,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,[490351126213007]
4,9607187010,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,[490351123023021]
...,...,...,...,...,...
9290,9048067208,"676 LAVA POINTE DR #50, SAINT GEORGE, UT",37.076327,-113.599085,[490532715021002]
9291,9342157005,"61 E GORDON AVE, SALT LAKE CITY, UT",40.683543,-111.889893,[490351116012016]
9292,9527478402,"756 S 300 W, Salt Lake City, UT",40.752633,-111.900108,[490351140003025]
9293,9736917000,"412 E. 3200 N., LEHI, UT",40.427645,-111.842536,[490490102163000]


Remove brackets

In [13]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,4440597205,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,490351140002011
1,5333267308,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,490111270024066
2,7850817707,"1055 North 500 West, Provo, UT",40.248562,-111.668402,490490020013000
3,8803828610,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,490351126213007
4,9607187010,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,490351123023021
...,...,...,...,...,...
9290,9048067208,"676 LAVA POINTE DR #50, SAINT GEORGE, UT",37.076327,-113.599085,490532715021002
9291,9342157005,"61 E GORDON AVE, SALT LAKE CITY, UT",40.683543,-111.889893,490351116012016
9292,9527478402,"756 S 300 W, Salt Lake City, UT",40.752633,-111.900108,490351140003025
9293,9736917000,"412 E. 3200 N., LEHI, UT",40.427645,-111.842536,490490102163000


In [14]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
603,3842137205,"1337 N 750 E, UT, UT",30.069203,-99.118273,NaN
9239,5601547110,"1750 E. Hwy 40, VERNAL, UT",39.001899,-92.733623,NaN


Parse for Merge

In [15]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,4440597205,490351140002011
1,5333267308,490111270024066
2,7850817707,490490020013000
3,8803828610,490351126213007
4,9607187010,490351123023021
...,...,...
9290,9048067208,490532715021002
9291,9342157005,490351116012016
9292,9527478402,490351140003025
9293,9736917000,490490102163000


Read in base data

In [16]:
#if from geocode file
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_ut.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head()

#if from FIPS file
#fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/UT_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,886823,886823,886823,4440597205,4/27/20,883,PPP,ALLISON BAVER ENTERTAINMENT LLC,6219 S. Saunter Lane,SALT LAKE CITY,...,999 Other,5121 Motion Picture and Video Industries,5121.0,UTAH DISTRICT OFFICE,e $5-10 million,250 to 499,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,NaN
1,886824,886824,886824,5333267308,4/30/20,883,PPP,"INTERMOUNTAIN EMPLOYMENT SERVICE, INC.",450 East 1000 North,NORTH SALT LAKE,...,999 Other,5613 Employment Services,5613.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,NaN
2,886825,886825,886825,7850817707,5/1/20,883,PPP,CENTRAL UTAH CLINIC PC,1055 North 500 West,Provo,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"1055 North 500 West, Provo, UT",40.248562,-111.668402,NaN
3,886826,886826,886826,8803828610,3/25/21,883,PPP,WOLVERINE FUELS HOLDING LLC,9815 S Monroe St Ste 203,Sandy,...,2121 Coal Mining,2121 Coal Mining,2121.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,NaN
4,886827,886827,886827,9607187010,4/9/20,883,PPP,"SIZZLING CAESARS, LLC",348 E WINCHESTER ST,SALT LAKE CITY,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,NaN


Merge with MainFrame

In [17]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,886823,886823,886823,4440597205,4/27/20,883,PPP,ALLISON BAVER ENTERTAINMENT LLC,6219 S. Saunter Lane,SALT LAKE CITY,...,5121 Motion Picture and Video Industries,5121.0,UTAH DISTRICT OFFICE,e $5-10 million,250 to 499,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,NaN,490351140002011
1,886824,886824,886824,5333267308,4/30/20,883,PPP,"INTERMOUNTAIN EMPLOYMENT SERVICE, INC.",450 East 1000 North,NORTH SALT LAKE,...,5613 Employment Services,5613.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,NaN,490111270024066
2,886825,886825,886825,7850817707,5/1/20,883,PPP,CENTRAL UTAH CLINIC PC,1055 North 500 West,Provo,...,6221 General Medical and Surgical Hospitals,6221.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"1055 North 500 West, Provo, UT",40.248562,-111.668402,NaN,490490020013000
3,886826,886826,886826,8803828610,3/25/21,883,PPP,WOLVERINE FUELS HOLDING LLC,9815 S Monroe St Ste 203,Sandy,...,2121 Coal Mining,2121.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,NaN,490351126213007
4,886827,886827,886827,9607187010,4/9/20,883,PPP,"SIZZLING CAESARS, LLC",348 E WINCHESTER ST,SALT LAKE CITY,...,7225 Restaurants and Other Eating Places,7225.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,NaN,490351123023021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9290,896113,896113,896113,9048067208,4/28/20,883,PPP,PRO TOWER MAINTENANCE LLC,676 LAVA POINTE DR #50,SAINT GEORGE,...,2383 Building Finishing Contractors,2383.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"676 LAVA POINTE DR #50, SAINT GEORGE, UT",37.076327,-113.599085,NaN,490532715021002
9291,896114,896114,896114,9342157005,4/9/20,883,PPP,"EDWARDS AUTOMOTIVE, INC",61 E GORDON AVE,SALT LAKE CITY,...,8111 Automotive Repair and Maintenance,8111.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"61 E GORDON AVE, SALT LAKE CITY, UT",40.683543,-111.889893,NaN,490351116012016
9292,896115,896115,896115,9527478402,2/17/21,883,PPS,RUBYSNAP LLC,756 S 300 W,Salt Lake City,...,3118 Bakeries and Tortilla Manufacturing,3118.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"756 S 300 W, Salt Lake City, UT",40.752633,-111.900108,NaN,490351140003025
9293,896116,896116,896116,9736917000,4/9/20,883,PPP,"CODY EKKER CONSTRUCTION, INC.",412 E. 3200 N.,LEHI,...,2362 Nonresidential Building Construction,2362.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"412 E. 3200 N., LEHI, UT",40.427645,-111.842536,NaN,490490102163000


<b>Fill FIPS Columns - if FIPS file

In [18]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,886823,886823,886823,4440597205,4/27/20,883,PPP,ALLISON BAVER ENTERTAINMENT LLC,6219 S. Saunter Lane,SALT LAKE CITY,...,5121.0,UTAH DISTRICT OFFICE,e $5-10 million,250 to 499,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,NaN,490351140002011,490351140002011
1,886824,886824,886824,5333267308,4/30/20,883,PPP,"INTERMOUNTAIN EMPLOYMENT SERVICE, INC.",450 East 1000 North,NORTH SALT LAKE,...,5613.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,NaN,490111270024066,490111270024066
2,886825,886825,886825,7850817707,5/1/20,883,PPP,CENTRAL UTAH CLINIC PC,1055 North 500 West,Provo,...,6221.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"1055 North 500 West, Provo, UT",40.248562,-111.668402,NaN,490490020013000,490490020013000
3,886826,886826,886826,8803828610,3/25/21,883,PPP,WOLVERINE FUELS HOLDING LLC,9815 S Monroe St Ste 203,Sandy,...,2121.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,NaN,490351126213007,490351126213007
4,886827,886827,886827,9607187010,4/9/20,883,PPP,"SIZZLING CAESARS, LLC",348 E WINCHESTER ST,SALT LAKE CITY,...,7225.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,NaN,490351123023021,490351123023021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9290,896113,896113,896113,9048067208,4/28/20,883,PPP,PRO TOWER MAINTENANCE LLC,676 LAVA POINTE DR #50,SAINT GEORGE,...,2383.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"676 LAVA POINTE DR #50, SAINT GEORGE, UT",37.076327,-113.599085,NaN,490532715021002,490532715021002
9291,896114,896114,896114,9342157005,4/9/20,883,PPP,"EDWARDS AUTOMOTIVE, INC",61 E GORDON AVE,SALT LAKE CITY,...,8111.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"61 E GORDON AVE, SALT LAKE CITY, UT",40.683543,-111.889893,NaN,490351116012016,490351116012016
9292,896115,896115,896115,9527478402,2/17/21,883,PPS,RUBYSNAP LLC,756 S 300 W,Salt Lake City,...,3118.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"756 S 300 W, Salt Lake City, UT",40.752633,-111.900108,NaN,490351140003025,490351140003025
9293,896116,896116,896116,9736917000,4/9/20,883,PPP,"CODY EKKER CONSTRUCTION, INC.",412 E. 3200 N.,LEHI,...,2362.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"412 E. 3200 N., LEHI, UT",40.427645,-111.842536,NaN,490490102163000,490490102163000


Drop Extra FIPS Columns

In [19]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,886823,886823,886823,4440597205,4/27/20,883,PPP,ALLISON BAVER ENTERTAINMENT LLC,6219 S. Saunter Lane,SALT LAKE CITY,...,999 Other,5121 Motion Picture and Video Industries,5121.0,UTAH DISTRICT OFFICE,e $5-10 million,250 to 499,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,490351140002011
1,886824,886824,886824,5333267308,4/30/20,883,PPP,"INTERMOUNTAIN EMPLOYMENT SERVICE, INC.",450 East 1000 North,NORTH SALT LAKE,...,999 Other,5613 Employment Services,5613.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,490111270024066
2,886825,886825,886825,7850817707,5/1/20,883,PPP,CENTRAL UTAH CLINIC PC,1055 North 500 West,Provo,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"1055 North 500 West, Provo, UT",40.248562,-111.668402,490490020013000
3,886826,886826,886826,8803828610,3/25/21,883,PPP,WOLVERINE FUELS HOLDING LLC,9815 S Monroe St Ste 203,Sandy,...,2121 Coal Mining,2121 Coal Mining,2121.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,490351126213007
4,886827,886827,886827,9607187010,4/9/20,883,PPP,"SIZZLING CAESARS, LLC",348 E WINCHESTER ST,SALT LAKE CITY,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,490351123023021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9290,896113,896113,896113,9048067208,4/28/20,883,PPP,PRO TOWER MAINTENANCE LLC,676 LAVA POINTE DR #50,SAINT GEORGE,...,999 Other,2383 Building Finishing Contractors,2383.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"676 LAVA POINTE DR #50, SAINT GEORGE, UT",37.076327,-113.599085,490532715021002
9291,896114,896114,896114,9342157005,4/9/20,883,PPP,"EDWARDS AUTOMOTIVE, INC",61 E GORDON AVE,SALT LAKE CITY,...,999 Other,8111 Automotive Repair and Maintenance,8111.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"61 E GORDON AVE, SALT LAKE CITY, UT",40.683543,-111.889893,490351116012016
9292,896115,896115,896115,9527478402,2/17/21,883,PPS,RUBYSNAP LLC,756 S 300 W,Salt Lake City,...,999 Other,3118 Bakeries and Tortilla Manufacturing,3118.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"756 S 300 W, Salt Lake City, UT",40.752633,-111.900108,490351140003025
9293,896116,896116,896116,9736917000,4/9/20,883,PPP,"CODY EKKER CONSTRUCTION, INC.",412 E. 3200 N.,LEHI,...,999 Other,2362 Nonresidential Building Construction,2362.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"412 E. 3200 N., LEHI, UT",40.427645,-111.842536,490490102163000


In [20]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
603,887426,887426,887426,3842137205,4/27/20,883,PPP,"LEGALLY MINE, LLC",1337 N 750 E,UT,...,999 Other,5411 Legal Services,5411.0,UTAH DISTRICT OFFICE,c $1-2 million,50 to 99,"1337 N 750 E, UT, UT",30.069203,-99.118273,NaN
9239,896062,896062,896062,5601547110,4/13/20,883,PPP,UDELLS CABINETS INC,1750 E. Hwy 40,VERNAL,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1750 E. Hwy 40, VERNAL, UT",39.001899,-92.733623,NaN


Convert to File

In [21]:
fips_UT = fips_merge_drop

In [22]:
fips_UT.to_csv("../../data/state_data/geo/geo_fips/UT_fips_scraped.csv") 

Review & Compare

In [23]:
geo_ut_fips = pd.read_csv("../../data/state_data/geo/geo_fips/UT_fips_scraped.csv") 
geo_ut_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,886823,886823,886823,4440597205,4/27/20,883,PPP,ALLISON BAVER ENTERTAINMENT LLC,6219 S. Saunter Lane,...,999 Other,5121 Motion Picture and Video Industries,5121.0,UTAH DISTRICT OFFICE,e $5-10 million,250 to 499,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,4.903511e+14
1,1,886824,886824,886824,5333267308,4/30/20,883,PPP,"INTERMOUNTAIN EMPLOYMENT SERVICE, INC.",450 East 1000 North,...,999 Other,5613 Employment Services,5613.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,4.901113e+14
2,2,886825,886825,886825,7850817707,5/1/20,883,PPP,CENTRAL UTAH CLINIC PC,1055 North 500 West,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"1055 North 500 West, Provo, UT",40.248562,-111.668402,4.904900e+14
3,3,886826,886826,886826,8803828610,3/25/21,883,PPP,WOLVERINE FUELS HOLDING LLC,9815 S Monroe St Ste 203,...,2121 Coal Mining,2121 Coal Mining,2121.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,4.903511e+14
4,4,886827,886827,886827,9607187010,4/9/20,883,PPP,"SIZZLING CAESARS, LLC",348 E WINCHESTER ST,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,UTAH DISTRICT OFFICE,e $5-10 million,500 or more,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,4.903511e+14


In [24]:
##### fill in missing Lat, Long, FIPS
geo_ut_fips.loc[geo_ut_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
603,603,887426,887426,887426,3842137205,4/27/20,883,PPP,"LEGALLY MINE, LLC",1337 N 750 E,...,999 Other,5411 Legal Services,5411.0,UTAH DISTRICT OFFICE,c $1-2 million,50 to 99,"1337 N 750 E, UT, UT",30.069203,-99.118273,NaN
9239,9239,896062,896062,896062,5601547110,4/13/20,883,PPP,UDELLS CABINETS INC,1750 E. Hwy 40,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,UTAH DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1750 E. Hwy 40, VERNAL, UT",39.001899,-92.733623,NaN
